# Using Machine Learning to Find Similar NBA Players

In [1]:
# Import packages

import pprint
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import scikit-learn moduldes
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree

## Import Data

In [2]:
# User path to data file
file_mini = "/Users/mini/Documents/nba_data/data_files/player_data_1981-2017.csv"
file_tiny = "/Users/benjaminxiao/Documents/nba_data/data_files/player_data_1981-2017.csv"
data = pd.read_csv(file_tiny)

In [3]:
data.columns.values

array(['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP_tot',
       'FG_tot', 'FGA_tot', 'FG_perc', '3P_tot', '3PA_tot', '3P_perc',
       '2P_tot', '2PA_tot', '2P_perc', 'eFG_perc', 'FT_tot', 'FTA_tot',
       'FT_perc', 'ORB_tot', 'DRB_tot', 'TRB_tot', 'AST_tot', 'STL_tot',
       'BLK_tot', 'TOV_tot', 'PF_tot', 'PTS_tot', 'MP_per_G', 'FG_per_G',
       'FGA_per_G', '3P_per_G', '3PA_per_G', '2P_per_G', '2PA_per_G',
       'FT_per_G', 'FTA_per_G', 'ORB_per_G', 'DRB_per_G', 'TRB_per_G',
       'AST_per_G', 'STL_per_G', 'BLK_per_G', 'TOV_per_G', 'PF_per_G',
       'PTS_per_G', 'FG_per_36m', 'FGA_per_36m', '3P_per_36m',
       '3PA_per_36m', '2P_per_36m', '2PA_per_36m', 'FT_per_36m',
       'FTA_per_36m', 'ORB_per_36m', 'DRB_per_36m', 'TRB_per_36m',
       'AST_per_36m', 'STL_per_36m', 'BLK_per_36m', 'TOV_per_36m',
       'PF_per_36m', 'PTS_per_36m', 'PER', 'TS_perc', '3PAr', 'FTr',
       'ORB_perc', 'DRB_perc', 'TRB_perc', 'AST_perc', 'STL_perc',
       'BLK_perc', 'TOV_perc', '

In [4]:
data.shape

(15234, 106)

## Data Cleaning

In [5]:
# Subset data to G >= 27, MP_tot >= 450
data = data[(data.G >= 27) & (data.MP_tot >= 450)]

I had to lower the minute limit to 450 to include Speedy Claxton

In [6]:
tot_list = data.columns[8:30]

In [7]:
per_100p_list = data.columns[-20:-3]

In [8]:
drop_cols = [] # Store columns to drop
for col in tot_list: # Drop season totals
    drop_cols.append(col)
    
for col in per_100p_list: # Drop per 100 possessions stats
    drop_cols.append(col)

The extra data was making the player query inaccurate from what I could tell.

In [9]:
# Drop unwanted columns
data.drop(drop_cols, axis = 1, inplace=True)

In [10]:
# Reset indices for querying later
data = data.reset_index()

## Data Preprocessing

In [11]:
# Assign feature and target values
# Exclude things that are labels and not useful statistics
x = data.drop(['Season', 'Player', 'Pos', 'Tm', 'G', 'GS', 'Rounded_Pos'], axis = 1)
y = data['Pos']

In [12]:
x.columns.values

array(['index', 'Age', 'MP_tot', 'MP_per_G', 'FG_per_G', 'FGA_per_G',
       '3P_per_G', '3PA_per_G', '2P_per_G', '2PA_per_G', 'FT_per_G',
       'FTA_per_G', 'ORB_per_G', 'DRB_per_G', 'TRB_per_G', 'AST_per_G',
       'STL_per_G', 'BLK_per_G', 'TOV_per_G', 'PF_per_G', 'PTS_per_G',
       'FG_per_36m', 'FGA_per_36m', '3P_per_36m', '3PA_per_36m',
       '2P_per_36m', '2PA_per_36m', 'FT_per_36m', 'FTA_per_36m',
       'ORB_per_36m', 'DRB_per_36m', 'TRB_per_36m', 'AST_per_36m',
       'STL_per_36m', 'BLK_per_36m', 'TOV_per_36m', 'PF_per_36m',
       'PTS_per_36m', 'PER', 'TS_perc', '3PAr', 'FTr', 'ORB_perc',
       'DRB_perc', 'TRB_perc', 'AST_perc', 'STL_perc', 'BLK_perc',
       'TOV_perc', 'USG_perc', 'OWS', 'DWS', 'WS', 'WS_per_48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'MP', 'ORtg', 'DRtg'], dtype=object)

In [13]:
# Scale data for dimensionality reduction
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

## Search with KDTree

In [14]:
# Queries KDTree for an array of 30 similar players 
def kdtree_search(input_data, player_index):
    tree = KDTree(input_data, leaf_size=40, metric='euclidean')
    dist, ind = tree.query([input_data[player_index]], k = 30)
    return ind # Return array of 30 similar players

In [15]:
# Get 3 similar players in array of indices
# Limits returned 3 players to seasons after 2010-11
def get_three_players(ind):
    # Gets 8 recent, similar players in order of shortest distance
    subset_recent_players = ind[0] >= 7745 # Get players after 2010-11 season
    recent_players = ind[0][subset_recent_players]
    # Get first three players in array
    return recent_players[:8]

In [16]:
# Get the indices of our target players
def get_player_indices(df, target_player, target_season):
    return df.index[(df['Player'] == target_player) & (df['Season'] == target_season)][0]

In [17]:
# Inputs similar players into a list of tuples
def similar_players(df, array):
    data = []
    for i in array:
        player_name = df['Player'].iloc[i] # Similar player's name
        player_season = df['Season'].iloc[i] # Similar player's season registered
        player_team = df['Tm'].iloc[i] # Similar player's team that season
        data.append(tuple((player_name, player_season, player_team)))
    # Return list of tuples including player name, team, and season
    return data

In [18]:
"""
Takes a DataFrame, target player name, and target player season, and outputs a dictionary. 
The dictionary has the target players as keys, and tuples of similar player names, seasons registerd, teams that year.
"""
def search_for_similar_players(df, target_player, target_season, storage_dict):
    idx = get_player_indices(df, target_player, target_season)
    array = kdtree_search(x_scaled, idx)
    similar_player_indices = get_three_players(array)
    # Store to storage_dict
    storage_dict[target_player] = similar_players(df, similar_player_indices)

In [45]:
# Takes the stored data and removes data points that output the same player as our target player
def rm_same_player(players_dict, players_list):
    for player in players_list:
        #print(player)
        #print(players_dict[player]) # Access list of tuples
        for i, similar_player in enumerate(players_dict[player]):
            #print(similar_player[0]) # access each player
            #print(similar_player) # access tuples
            if similar_player[0] == player:
                players_dict[player].pop(i)

## Gathering Similar Players for Two Championship Teams

In [20]:
"""
1. Create roster list of 2003 Spurs minus Tim Duncan.
2. Create another roster list of the 2011 Mavericks minus Dirk.
3. These are the major contributing players for the majority of the playoffs. 
"""

spurs_roster_list = ['Tony Parker', 'David Robinson*', 'Stephen Jackson', 'Manu Ginobili', \
              'Malik Rose', 'Speedy Claxton', 'Bruce Bowen']

mavs_roster_list = ['Jason Terry', 'Shawn Marion', 'Tyson Chandler', 'J.J. Barea', \
                   'Jason Kidd', 'DeShawn Stevenson', 'Peja Stojakovic']

spurs_season = '2002-03'
mavs_season = '2010-11'

# Create an empty dictionary with roster list
similar_players_spurs = dict((el,None) for el in spurs_roster_list)
similar_players_mavs = dict((el, None) for el in mavs_roster_list)

In [49]:
# Call function for 2003 Spurs
for name in spurs_roster_list:
    search_for_similar_players(data, name, spurs_season, similar_players_spurs)
    
# Remove the same players from the list
rm_same_player(similar_players_spurs, spurs_roster_list)

In [53]:
# Call function for 2011 Mavs
for name in mavs_roster_list:
    search_for_similar_players(data, name, mavs_season, similar_players_mavs)
    
# Remove the same players from the list
rm_same_player(similar_players_mavs, mavs_roster_list)

In [50]:
pprint.pprint(similar_players_spurs)

{'Bruce Bowen': [('Anthony Parker', '2009-10', 'CLE'),
                 ('Jared Dudley', '2015-16', 'WAS'),
                 ('Shane Battier', '2010-11', 'TOT'),
                 ('Shane Battier', '2008-09', 'HOU'),
                 ('Shane Battier', '2009-10', 'HOU'),
                 ('Courtney Lee', '2014-15', 'MEM'),
                 ('Ime Udoka', '2006-07', 'POR')],
 'David Robinson*': [('Tim Duncan', '2015-16', 'SAS'),
                     ('Samuel Dalembert', '2011-12', 'HOU'),
                     ('Ian Mahinmi', '2015-16', 'IND'),
                     ('Emeka Okafor', '2010-11', 'NOH'),
                     ('Marcus Camby', '2008-09', 'LAC'),
                     ('DeAndre Jordan', '2012-13', 'LAC')],
 'Malik Rose': [('Nene Hilario', '2012-13', 'WAS'),
                ('Luis Scola', '2007-08', 'HOU')],
 'Manu Ginobili': [('James Harden', '2009-10', 'OKC'),
                   ('Trevor Ariza', '2008-09', 'LAL'),
                   ('Mario Chalmers', '2013-14', 'MIA')],
 'Speedy 

In [54]:
pprint.pprint(similar_players_mavs)

{'DeShawn Stevenson': [('DeShawn Stevenson', '2012-13', 'ATL'),
                       ('Damjan Rudez', '2014-15', 'IND'),
                       ('Mickael Pietrus', '2010-11', 'TOT'),
                       ('Keith Bogans', '2012-13', 'BRK'),
                       ('Eddie House', '2010-11', 'MIA'),
                       ('James Jones', '2009-10', 'MIA'),
                       ('Anthony Tolliver', '2014-15', 'TOT')],
 'J.J. Barea': [('Jordan Crawford', '2013-14', 'TOT'),
                ('J.J. Barea', '2008-09', 'DAL'),
                ('Greivis Vasquez', '2013-14', 'TOT'),
                ('Randy Foye', '2009-10', 'WAS'),
                ('D.J. Augustin', '2014-15', 'TOT')],
 'Jason Kidd': [('Jason Kidd', '2009-10', 'DAL'),
                ('Jason Kidd', '2011-12', 'DAL'),
                ('Nicolas Batum', '2014-15', 'POR'),
                ('Andre Iguodala', '2013-14', 'GSW')],
 'Jason Terry': [('Jason Terry', '2009-10', 'DAL'),
                 ('Raymond Felton', '2012-13', 'NYK'

### Next Step

* How to visualize this data for a blog post?
* Drop players who are the same as target player